# 2.1 Solving PDEs with finite elements in NGSolve


In [3]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

geo = SplineGeometry()
geo.AddCircle((0.5,0.5), 0.5, bc="circle")

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
mesh.Curve(3)

In [4]:
fes = H1(mesh, order=3, dirichlet="circle")
u = fes.TrialFunction()
w = fes.TestFunction()

In [5]:
X = CoefficientFunction((x,y,z))

In [6]:
L = LinearForm(fes)
# 2*y*(1-y)+2*x*(1-x)
#f1 = (2*X[1]*(1-X[1])+2*X[0]*(1-X[0]))
f1 = X[0]**2
L += f1 * w * dx

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(w)*dx

In [7]:
a.Assemble()
L.Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") * L.vec

Draw(gfu,mesh,"state")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

# 3.1.1 Shape differentiation of unconstrained volume integrals
f (x1, x2) = (x1 − 0.5)2/a2 + (x2 −0.5)2/b2 − R2

with a = 1.3, b = 1/a and R = 0.5

In [8]:
f = ((X[0]-0.5)/1.3)**2+(1.3*(X[1]-0.5))**2-0.5**2
F = Id(2)
G_f = f * Det(F) * dx

In [9]:
VEC = VectorH1(mesh, order=1, dirichlet="")
V = VEC.TestFunction()

dJOmega_f = LinearForm(VEC)
dJOmega_f += G_f.Diff(X, V)
dJOmega_f += G_f.Diff(F, grad(V))

# 3.1.2 Shape differentiation of unconstrained boundary integrals

In [10]:
G_f_bnd = f * Det(F) * Norm(Inv(F).trans * specialcf.normal(2)) * ds
dJOmega_f_bnd = LinearForm(VEC)
dJOmega_f_bnd += G_f_bnd.Diff(X, V)
dJOmega_f_bnd += G_f_bnd.Diff(F, grad(V).Trace())

# 3.2 Second-order shape derivatives

In [11]:
d2JOmega_f = BilinearForm(VEC)
W = VEC.TrialFunction()
d2JOmega_f += (G_f.Diff(X,W)+G_f.Diff(F,grad(W))).Diff(X,V)
d2JOmega_f += (G_f.Diff(X,W)+G_f.Diff(F,grad(W))).Diff(F,grad(V))

# 3.2 2nd order boundary integrals

In [12]:
d2JOmega_f_bnd = BilinearForm(VEC)
d2JOmega_f_bnd += (G_f_bnd.Diff(X,W)+G_f_bnd.Diff(F,grad(W).Trace())).Diff(X,V)
d2JOmega_f_bnd += (G_f_bnd.Diff(X,W)+G_f_bnd.Diff(F,grad(W).Trace())).Diff(F,grad(V).Trace())

# 4 Semi-automatic shape differentiation with PDE constraints
ud = x*(x-1)*y*(1-y) 

Cost = (u-ud)² * Det(f) * dx (31)

In [13]:
ud = X[0]*(1-X[0])*X[1]*(1-X[1])
def Cost(u):
    return (u-ud)**2 * Det(F) * dx

# solve adjoint equation
gfp = GridFunction(fes)
dCostdu = LinearForm(fes)
dCostdu += Cost(gfu).Diff(gfu, w)
dCostdu.Assemble()
gfp.vec.data = -a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky").T * dCostdu.vec
Draw(gfp, mesh, "adjoint")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [14]:
def Equation(u,w): #(31)
    return ((Inv(F).trans * grad(u)) * (Inv(F).trans * grad(w)) - f*w)*Det(F)*dx

G_pde = Cost(gfu) + Equation(gfu, gfp)

dFOmega_pde = LinearForm(VEC)
dFOmega_pde += G_pde.Diff(X,V)
dFOmega_pde += G_pde.Diff(F, grad(V))

# 4.3 Second-order shape derivative

In [15]:
X2 = FESpace([fes, fes])
dsu, dsp = X2.TrialFunction()
uTest, pTest = X2.TestFunction()
gfV = GridFunction(VEC)
gfW = GridFunction(VEC)
gfV.Set((X[0]*X[0]*X[1]*exp(X[1]),X[1]*X[1]*X[0]*exp(X[0])))
gfW.Set((X[1]*X[1]*X[0]*exp(X[0]), X[0]*X[0]*X[1]*exp(X[1])))

In [16]:
shapeHessLag2 = BilinearForm(X2)
shapeGradLag2 = LinearForm(X2)

shapeHessLag2 += (G_pde.Diff(gfu, uTest)).Diff(gfu, dsu) #block(1,1)
shapeHessLag2 += (G_pde.Diff(gfu, uTest)).Diff(gfp, dsp) #block(1,2)
shapeHessLag2 += (G_pde.Diff(gfu, pTest)).Diff(gfp, dsu) #block(2,1)

#line 1
shapeGradLag2 += (G_pde.Diff(gfu, uTest)).Diff(F, grad(gfV))
shapeGradLag2 += (G_pde.Diff(gfu, uTest)).Diff(X, gfV)

#line 2
shapeGradLag2 += (G_pde.Diff(gfp, pTest)).Diff(F, grad(gfV))
shapeGradLag2 += (G_pde.Diff(gfp, pTest)).Diff(X, gfV)

gfCombined2 = GridFunction(X2)
shapeHessLag2.Assemble()
shapeGradLag2.Assemble()
gfCombined2.vec.data = shapeHessLag2.mat.Inverse(X2.FreeDofs(), inverse="pardiso") * shapeGradLag2.vec

gfdsu = GridFunction(fes)
gfdsp = GridFunction(fes)
gfdsu.vec.data = gfCombined2.components[0].vec
gfdsp.vec.data = gfCombined2.components[1].vec

Draw(gfdsu, mesh, "dsu")
Draw(gfdsp, mesh, "dsp")

# 5. Fully automated shape differentiation

In [17]:
G_f_0 = f*dx
dJOmega_f_0 = LinearForm(VEC)
dJOmega_f_0 += G_f_0.DiffShape(V)

In [18]:
def Cost_0(u):
    return (u-ud)**2*dx

def Equation_0(u,w):
    return (grad(u)*grad(w)-f1*w)*dx

G_pde_0 = Cost_0(gfu) + Equation_0(gfu, gfp)

dJOmega_pde_0 = LinearForm(VEC)
dJOmega_pde_0 += G_pde_0.DiffShape(V)

In [22]:
#mesh.UnsetDeformation()
gfset = GridFunction(VEC)
Draw(gfset, mesh, "gfset")      
SetVisualization(deformation=True)
# x²y, y²x
gfset.Set((X[0]*X[0]*X[1], X[1]*X[1]*X[0]))
mesh.SetDeformation(gfset)
Redraw()

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [110]:
def solvePDE():
    a.Assemble()
    L.Assemble()
    dCostdu.Assemble()
    
    inva = a.mat.Inverse(fes.FreeDofs(), inverse = "sparsecholesky")
    gfu.vec.data = inva*L.vec
    gfp.vec.data = -inva.T * dCostdu.vec
    
solvePDE()
Draw(gfset, mesh, "gfset")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [1]:
def moveNGmesh2D(displ, mesh):
    for p in mesh.ngmesh.Points():
        v = displ(mesh(p[0], p[1]))
        p[0] += v[0]
        p[1] += v[1]
    mesh.ngmesh.Update()

moveNGmesh2D((X[0]*0.1,X[1]), gfset)
# ngmesh.OptimizeMesh2d()

NameError: name 'X' is not defined